In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
from sklearn import linear_model
from scipy.stats import multivariate_normal, spearmanr, kendalltau
from datetime import date



sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

<h3>Часть 1. Загрузка и отбор данных</h3>

In [2]:
import json, pickle

tournaments = pickle.load(open('chgk/tournaments.pkl', 'rb'))
results = pickle.load(open('chgk/results.pkl', 'rb'))
players = pickle.load(open('chgk/players.pkl', 'rb'))

In [3]:
train = {k : v['name'] for k,v in tournaments.items() if v['dateStart'][:4] == '2019' and
        len(results[k]) > 0 and
        len([p['player']['id'] for p in results[k][0]['teamMembers']]) > 0 and
        'mask' in results[k][0].keys() and
        results[k][0]['mask'] is not None }# and
        #len(results[k][0]['mask']) == sum(tournaments[k]['questionQty'].values())}

In [4]:
n_train_tournaments = len(train.keys())
n_train_tournaments

674

In [5]:
test = {k : v['name'] for k,v in tournaments.items() if v['dateStart'][:4] == '2020' and
        len(results[k]) > 0 and
        len([p['player']['id'] for p in results[k][0]['teamMembers']]) > 0 and
        'mask' in results[k][0].keys() and
        results[k][0]['mask'] is not None} #and
        #len(results[k][0]['mask']) == sum(tournaments[k]['questionQty'].values())}

In [6]:
len(test.keys())

173

Получили 674 турнира в тренировочном наборе и 173 в тестовом.

<h3>Часть 2. Рейтинг-лист игрков. Baseline - модель</h3>

<h5>Извелечение гипотетически полезных данных</h5>

Для обучения baseline-модели было принято решение использовать вопросные результаты ответов игроков из Train-части(считаем, как и было предложено для упрощения, что игрок ответил верно, если его команда ответила верно). В качестве характеристик игрока вытащим из данных долю турниров, в которых он принял участие, долю турниров, в которых его команда заняла первое, второе или третье места, долю правильных ответов, а также среднюю относительную позицию его команды в турнире. Вопросы будем описывать также двумя характеристиками: популярность турнира(нормированное число команд на отрезок от 0 до 1) и сложность вопроса(доля команд турнира, которая на него не ответила). Target-переменная - взят вопрос или нет. 

In [7]:
def get_questions_complexity(tourn_key):
    
    max_mask_len = 0
    for j in range(len(results[tourn_key])):
        if results[tourn_key][0]['mask'] is not None and len(results[tourn_key][0]['mask']) > max_mask_len:
            max_mask_len = len(results[tourn_key][0]['mask'])
    
    
    not_answered = [0 for i in range(max_mask_len)]
    check = [True for i in range(max_mask_len)]
    bad_teams = 0
    for team_ind in range(len(results[tourn_key])):
        if results[tourn_key][team_ind]['mask'] is None or len(results[tourn_key][team_ind]['mask']) != max_mask_len:
            bad_teams += 1
            continue
        for question_ind in range(len(results[tourn_key][team_ind]['mask'])):
            if results[tourn_key][team_ind]['mask'][question_ind] == '0' or results[tourn_key][team_ind]['mask'][question_ind] == '?':
                not_answered[question_ind] += 1
            elif results[tourn_key][team_ind]['mask'][question_ind] != '1' :
                #some problems with question
                check[question_ind] = False
    not_answered = np.array(not_answered)
    not_answered = not_answered[check] / (len(results[tourn_key]) - bad_teams)
    return max_mask_len, check, not_answered

In [8]:
def get_tournaments_popularities():
    all_popularities = {}
    for tourn_key in train.keys():
        all_popularities[tourn_key] = len(results[tourn_key])
    min_popularity = min(all_popularities.values())
    max_popularity = max(all_popularities.values())
    for tourn_key in train.keys():
        all_popularities[tourn_key] = (all_popularities[tourn_key]-min_popularity)/(max_popularity-min_popularity)
    return all_popularities

In [9]:
dict_top3 = {}
dict_participate = {}
dict_questions = {}
dict_avg_position = {}
tournament_popularities = get_tournaments_popularities()

for tourn_key in train.keys():
    
    max_mask_len, check, questions_complexity = get_questions_complexity(tourn_key)
    tournament_popularity = tournament_popularities[tourn_key]
    popularity_array = [tournament_popularity for i in range(len(questions_complexity))]
    
    for team_ind in range(len(results[tourn_key])):
        
        team_mask = results[tourn_key][team_ind]['mask']
        
        if team_mask is None or len(team_mask) != max_mask_len:
            continue
            
        team_results = []
        for i in range(len(team_mask)):
            if check[i] and team_mask[i] == '1':
                team_results.append(1)
            elif check[i]:
                team_results.append(0)
        
        
        for member_ind in range(len(results[tourn_key][team_ind]['teamMembers'])):
            player_id = results[tourn_key][team_ind]['teamMembers'][member_ind]['player']['id']
            
            if player_id in dict_participate.keys():
                dict_participate[player_id] += 1
            else:
                dict_participate[player_id] = 1
                
            if player_id in dict_avg_position.keys():
                dict_avg_position[player_id].append(1 - (team_ind + 1) / len(results[tourn_key]))
            else:
                dict_avg_position[player_id] = [1 - (team_ind + 1) / len(results[tourn_key])]
                
            if player_id in dict_top3.keys() and team_ind <= 2:
                dict_top3[player_id] += 1
            elif player_id not in dict_top3.keys() and team_ind <= 2:
                dict_top3[player_id] = 1
            elif player_id not in dict_top3.keys():
                dict_top3[player_id] = 0  
            
            
            if player_id in dict_questions.keys():
                dict_questions[player_id]['popularity'].extend(popularity_array)
                dict_questions[player_id]['complexity'].extend(questions_complexity)
                dict_questions[player_id]['result'].extend(team_results)
                dict_questions[player_id]['team'].extend([results[tourn_key][team_ind]['team']['id'] for _ in range(len(popularity_array))])
                dict_questions[player_id]['tournament'].extend([tourn_key for _ in range(len(popularity_array))])
                dict_questions[player_id]['members'].extend([len(results[tourn_key][team_ind]['teamMembers']) for _ in range(len(popularity_array))])
                dict_questions[player_id]['questions'].extend([r for r in range(len(popularity_array))])
            else:
                dict_questions[player_id] = {'popularity' : [],
                                            'complexity' : [],
                                            'result':[],
                                            'team':[],
                                            'tournament':[],
                                            'members':[],
                                            'questions':[]}
            

In [10]:
main_data_dict = {'tournament_id' : [],
                 'team_id' : [],
                 'n_members':[],
                 'player_id' : [],
                 'question_id' :[],
                 'games': [],
                 'top': [],
                 'correct' : [],
                 'position': [],
                 'popularity' : [],
                 'complexity' : [],
                 'result' : []}
for key in dict_participate.keys():
    n_questions = len(dict_questions[key]['result'])
    main_data_dict['tournament_id'].extend(dict_questions[key]['tournament'])
    main_data_dict['team_id'].extend(dict_questions[key]['team'])
    main_data_dict['n_members'].extend(dict_questions[key]['members'])
    main_data_dict['player_id'].extend([key for _ in range(n_questions)])
    main_data_dict['question_id'].extend(dict_questions[key]['questions'])
    main_data_dict['games'].extend([dict_participate[key] / len(train.keys()) for _ in range(n_questions)])
    main_data_dict['top'].extend([dict_top3[key] / len(train.keys()) for _ in range(n_questions)])
    main_data_dict['popularity'].extend(dict_questions[key]['popularity'])
    mean_pos = np.mean(dict_avg_position[key])
    main_data_dict['position'].extend([mean_pos for _ in range(n_questions)])
    main_data_dict['complexity'].extend(dict_questions[key]['complexity'])
    main_data_dict['result'].extend(dict_questions[key]['result'])
    correct_part = np.mean(dict_questions[key]['result'])
    main_data_dict['correct'].extend([correct_part for _ in range(len(dict_questions[key]['result']))])

/Users/gudkovivan/mupdated_miniconda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/gudkovivan/mupdated_miniconda/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
df = pd.DataFrame(data=main_data_dict)

In [12]:
df.head()

,tournament_id,team_id,n_members,player_id,question_id,games,top,correct,position,popularity,complexity,result
0,4973,45556,6,6212,0,0.118694,0.051929,0.708741,0.929924,0.508424,0.438169,1
1,4973,45556,6,6212,1,0.118694,0.051929,0.708741,0.929924,0.508424,0.196689,1
2,4973,45556,6,6212,2,0.118694,0.051929,0.708741,0.929924,0.508424,0.226874,1
3,4973,45556,6,6212,3,0.118694,0.051929,0.708741,0.929924,0.508424,0.460565,1
4,4973,45556,6,6212,4,0.118694,0.051929,0.708741,0.929924,0.508424,0.703019,1


In [13]:
df.tail()

,tournament_id,team_id,n_members,player_id,question_id,games,top,correct,position,popularity,complexity,result
15457986,6254,64197,11,223782,211,0.002967,0.0,0.490741,0.549232,0.418236,0.367876,1
15457987,6254,64197,11,223782,212,0.002967,0.0,0.490741,0.549232,0.418236,0.756477,0
15457988,6254,64197,11,223782,213,0.002967,0.0,0.490741,0.549232,0.418236,0.922280,0
15457989,6254,64197,11,223782,214,0.002967,0.0,0.490741,0.549232,0.418236,0.891192,0
15457990,6254,64197,11,223782,215,0.002967,0.0,0.490741,0.549232,0.418236,0.481865,1


In [14]:
from sklearn.linear_model import LogisticRegression

X_train, y_train = (df.loc[:, 'games':'complexity'], df.loc[:, 'result'])
clf = LogisticRegression().fit(X_train, y_train)

In [15]:
y_train.value_counts()

0    8476231
1    6981760
Name: result, dtype: int64

In [16]:
clf.score(X_train, y_train)

0.7588105724734864

In [17]:
clf.coef_

array([[ 0.15398907,  2.26489383,  2.70683066,  1.97179501,  0.08539625,
        -5.61193705]])

In [18]:
predicted_probas = clf.predict_proba(df.loc[:, 'games':'complexity'])[:,1]

In [19]:
df['rating'] = predicted_probas

In [20]:
rating = df.groupby('player_id', as_index=False).rating.mean().sort_values(by=['rating'], ascending = False)

In [21]:
rating.head()

,player_id,rating
27465,202413,0.868559
8840,101782,0.866693
26626,200556,0.856468
16427,162084,0.855036
25670,197605,0.854347


Сопоставим имена людей и id и выведем топ игроков baseline-модели

In [22]:
player_names = {}
for tourn_key in train.keys():
    for team_ind in range(len(results[tourn_key])):
        for member_ind in range(len(results[tourn_key][team_ind]['teamMembers'])):
            player_id = results[tourn_key][team_ind]['teamMembers'][member_ind]['player']['id']
            if player_id not in player_names.keys():
                player_names[player_id] = results[tourn_key][team_ind]['teamMembers'][member_ind]['player']['surname'] + ' ' + results[tourn_key][team_ind]['teamMembers'][member_ind]['player']['name'] + ' ' + results[tourn_key][team_ind]['teamMembers'][member_ind]['player']['patronymic']

In [23]:
names = []
ids = np.array(rating[['player_id']])
for i in range(len(ids)):
    names.append(player_names[ids[i][0]])
rating['names'] = names
rating.head(20)

,player_id,rating,names
27465,202413,0.868559,Каменских Мария Александровна
8840,101782,0.866693,Чистяков Антон Александрович
26626,200556,0.856468,Иващенко Фёдор Антонович
16427,162084,0.855036,Пустовий Виталий Дмитриевич
25670,197605,0.854347,Иловайская Елена
32957,212514,0.847638,Флимова Анна Александровна
2419,20056,0.846910,Мартьянов Евгений Михайлович
24754,194685,0.845630,Сергеев Антон
4119,34795,0.845630,Черкасова Анастасия
2368,19622,0.844541,Малькина Ирина Николаевна


<h3>Часть 3. Baseline - модель. Оценка качества</h3>

Попредсказываем результаты турниров на тестовом наборе, используя полученный на предыдущем шаге рейтинг-лист.  Чтобы получить рейтинг команды, усредним рейтинги игроков, входящих в команду. Чем выше будет итоговый рейтинг команды, тем лучше результат она покажет.  
Если игрок новый, будем считать, что его рейтинг равен медианному рейтингу всех игроков.
Для оценки качества будем считать средние ранговые корреляции Спирмена и Кендалла. 


In [24]:
median_rating = np.median(np.array(rating.rating))

spearman_corrs = []
kendall_corrs = []
for tourn_key in test.keys(): 
    tourn_results = {'ids':[],
                    'real_results':[],
                    'predicted_res':[]}
    team_ratings = []
    flag = True
    for team_ind in range(len(results[tourn_key])):
        tourn_results['ids'].append(results[tourn_key][team_ind]['team']['id'])
        tourn_results['real_results'].append(team_ind + 1)
        n_members = len(results[tourn_key][team_ind]['teamMembers'])
        if n_members == 0:
            flag = False
            break
        total_rating = 0
        for i in range(n_members):
            if results[tourn_key][team_ind]['teamMembers'][i]['player']['id'] in ids:
                total_rating += float(rating[rating.player_id.isin([results[tourn_key][team_ind]['teamMembers'][i]['player']['id']])].rating)
            else:
                total_rating += median_rating
        total_rating /= n_members
        team_ratings.append((results[tourn_key][team_ind]['team']['id'], total_rating))
    if not flag:
        continue
    team_ratings = sorted(team_ratings, key = lambda x: -x[1])
    pred_res = [None for _ in range(len(team_ratings))]
    for team_ind in range(len(team_ratings)):
         pred_res[tourn_results['ids'].index(team_ratings[team_ind][0])] = team_ind + 1
    tourn_results['predicted_res'] = pred_res
    
    coef, p = spearmanr(tourn_results['real_results'], tourn_results['predicted_res'], nan_policy = 'omit')
    if coef <= 1:
        spearman_corrs.append(coef)
    
    coef, p = kendalltau(tourn_results['real_results'], tourn_results['predicted_res'], nan_policy = 'omit')
    if coef <= 1:
        kendall_corrs.append(coef)

In [25]:
np.mean(spearman_corrs)

0.6633249941771249

In [26]:
np.mean(kendall_corrs)

0.4979736344898558

<h3>Часть 4. EM-алгоритм</h3>

В Baseline-модели мы никак не учитывали, кто из игроков реально ответил на вопрос. Попробуем учесть это. Для этого каждому игроку припишем рейтинг(вероятность ответа на конкретный вопрос), не зависящий от команды, а также вес игрока в команде, в которой он играет(зависит от команды). 
Логично будем считать, что в ходе обсуждения какой-то игрок предлагает свою версию ответа и все игроки сразу же соглашаются с ней. Вес конкретного игрока показывает, насколько вероятно, что именно с его версией согласятся. Таким образом, вес можно трактовать, как авторитет игрока.
Стоит отметить, что выбранной командой версия может быть либо правильной, либо неправильной. Напришивается использовать распределение Бернулли. 

Перейдем к формальному описанию $EM$-алгоритма. Рассмотрим один конкретный турнир. Для каждой команды-участницы $t$ введем скрытые переменные $Z_{kit}$, прчем $Z_{kit}=1$ тогда и только тогда, когда с версией ответа $k$-го игрока на $i$-й вопрос команды $t$ согласилась вся команда. Командный результат ответа на вопрос будем рассматривать как значение случайной величины с распределением Бернулли с параметром $p_k$, где $p_k$-рейтинг k-го игрока. 

Тогда на $E$-шаге мы будем вычислять вероятность того, что именно $k$-й игрок ответил на вопрос:

$\gamma_{ik} = \mathbb{P}(Z_{kit}=1) = \frac{w_{kt}p_k^{x_{it}}  (1 - p_k)^{x_{it}}}{\sum_{k = 1}^{K} w_{kt}p_k^{x_{it}} (1 - p_k)^{x_{it}}}$, где $K$ - число игроков в команде $t$, $w_{kt}$ - вес игрока в команде $t$, $x_{it}$ - 1 или 0, результат ответа на $i$-й вопрос командой $t$.

На $M$-шаге будем обновлять веса и рейтинг игроков:

$w_{kt}^{new} = \frac{1}{n} \sum_{i = 1}^{n} \gamma_{ik}$

$p_{k}^{new} = \frac{\sum_{i = 1}^{n} \gamma_{ik}x_i}{\sum_{i = 1}^{n} \gamma_{ik}}$

Последняя формула была получена из минимизации функционала $\sum_{i = 1}^{n}\sum_{k = 1}^{K} \gamma_{ik} ln(p_k^{x_{i}}  (1 - p_k)^{x_{i}})$ по $p_{k}$

Заметим, что мы обновляем рейтинги игроков в каждом турнире независимо, поэтому после $M$-шага будем усреднять новый рейтинг по игрокам. 

В качестве начального приближения весов(авторитетов игроков) команды возьмем числа, равные обратному числу игроков в команде. В качестве начального приближения рейтинга - бейзлайн-рейтинг


In [27]:
cur_ratings = {}

In [28]:
for i, row in rating.iterrows():
    cur_ratings[int(row.player_id)] = row.rating

In [29]:
df = df.loc[:, ['tournament_id', 'team_id', 'n_members', 'player_id', 'question_id', 'result']]
df.head()

,tournament_id,team_id,n_members,player_id,question_id,result
0,4973,45556,6,6212,0,1
1,4973,45556,6,6212,1,1
2,4973,45556,6,6212,2,1
3,4973,45556,6,6212,3,1
4,4973,45556,6,6212,4,1


In [30]:
rating = []
weight = []
for i, row in df.iterrows():
    weight.append(1 / row.n_members)
    rating.append(cur_ratings[row.player_id])
df['rating'] = rating
df['weight'] = weight
df.head()

,tournament_id,team_id,n_members,player_id,question_id,result,rating,weight
0,4973,45556,6,6212,0,1,0.703348,0.166667
1,4973,45556,6,6212,1,1,0.703348,0.166667
2,4973,45556,6,6212,2,1,0.703348,0.166667
3,4973,45556,6,6212,3,1,0.703348,0.166667
4,4973,45556,6,6212,4,1,0.703348,0.166667


Сохраним этот датасет на всякий случай, а то создается он довольно долго...

In [31]:
df.to_csv('em_data.csv')

In [32]:
df = df.drop(['n_members'], axis = 1)

In [33]:
df.head()

,tournament_id,team_id,player_id,question_id,result,rating,weight
0,4973,45556,6212,0,1,0.703348,0.166667
1,4973,45556,6212,1,1,0.703348,0.166667
2,4973,45556,6212,2,1,0.703348,0.166667
3,4973,45556,6212,3,1,0.703348,0.166667
4,4973,45556,6212,4,1,0.703348,0.166667


In [34]:
def e_step(df):
    gamma_hidden = np.zeros((len(set(df.question_id)), len(set(df.player_id))))
    cur_df = df.groupby(['question_id'])
    for g_idx, group in cur_df:
        if np.array(group.result)[0] == 1:
            arr = np.array(group.rating * group.weight)
            gamma_hidden[g_idx] = arr / np.sum(arr)
        else:
            arr = np.array((1 - group.rating) * group.weight)
            gamma_hidden[g_idx] = arr / np.sum(arr)
    return gamma_hidden

In [35]:
def m_step(gamma, df):
    w_new = np.mean(gamma, axis = 0)
    p_new = np.sum(gamma * np.array(df.groupby('player_id').result), axis = 1) / np.sum(gamma, axis = 1)
    ids = list(set(list(df.player_id)))
    result = pd.DataFrame()
    result['id'] = ids
    result['p_new'] = p_new
    result['w_new'] = w_new
    return result

In [36]:
def em_step(df):
    return m_step(e_step(df), df)

In [37]:
# df.groupby(['tournament_id', 'team_id']).apply(em_step)

А на закоментированной выше ячейке упало ядро... Доделать шаг с усреднением рейтинга по игрокам не вышло и проверить насколько лучше результат у ЕМ-алгоритма соответсвенно тоже(

<h3>Часть 5. Рейтинг-лист турниров по сложности вопросов</h3>

Отсортируем турниры по средней сложности вопросов. Сложностью одного конкретного вопроса в турнире считаем долю команд этого турнира, которые на него не ответили.

In [38]:
tournaments_complexity = {'ids' : [],
                         'complexity':[],
                         'name':[]}
for tourn_key in train.keys():
    max_mask_len, check, complexity = get_questions_complexity(tourn_key)
    tournaments_complexity['ids'].append(tourn_key)
    tournaments_complexity['complexity'].append(np.mean(complexity))
    tournaments_complexity['name'].append(tournaments[tourn_key]['name'])

In [39]:
complexity = pd.DataFrame(data=tournaments_complexity)

In [40]:
complexity.sort_values(by=['complexity'], ascending = False).head(10)

,ids,complexity,name
662,6149,1.000000,Чемпионат Санкт-Петербурга. Первая лига
387,5717,0.909402,Чемпионат Таджикистана
293,5599,0.844048,Чемпионат Туркменистана
575,5976,0.835648,Открытый Студенческий чемпионат Краснодарского...
666,6161,0.823188,Студенческий чемпионат Тюменской области
265,5564,0.815404,Молодёжный чемпионат Нижегородской области
368,5684,0.811429,Синхрон высшей лиги Москвы
142,5426,0.799074,Зимник
328,5637,0.798611,Чемпионат Кыргызстана
540,5928,0.796508,Угрюмый Ёрш


In [41]:
complexity.sort_values(by=['complexity'], ascending = False).tail(10)

,ids,complexity,name
327,5636,0.372685,Кубок Закарпатья
466,5827,0.370500,Шестой киевский марафон. Асинхрон
523,5899,0.365079,"Ничто, нигде, никогда"
544,5935,0.351754,Асинхрон по «Королю и Шуту»
648,6115,0.339506,Чемпионат МГУ. Высшая лига. Второй игровой день
245,5540,0.336420,Регулярный чемпионат МГУ. Высшая лига. Третий ...
395,5726,0.316308,Первый турнир имени Джоуи Триббиани
567,5963,0.259259,Асинхрон по South Park
597,6003,0.254115,Второй тематический турнир имени Джоуи Триббиани
153,5438,0.193985,Синхрон Лиги Разума


Даже не очень разбирающийся в ЧГК-шных турнирах человек может заметить, что названия турниров в начале списка более статусные и звучные: в топ10 по сложности вошли чемпионаты стран/городов/областей. В то время как самый низкую сложность получили асинхронные и тематические турниры. Результат вполне соответствует нашей интуиции.